In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [2]:
super_prob_cut = 0.7 #probability cut-off for test image belonging to highest probability superclass from training data (if lower, classified as novel)
sub_prob_cut = 0.4 #probability cut-off for test image belonging to highest probability subclass from training data (if lower, classified as novel)


In [3]:
from torchvision import models
backbone = models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 43.3MB/s]


In [4]:
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/NNDL_Project

Mounted at /content/drive
/content/drive/MyDrive/NNDL_Project


In [6]:
%%capture
! unzip -o train_shuffle.zip

In [7]:
%%capture
! unzip -o test_shuffle.zip

In [8]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [9]:
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True, drop_last = True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True, drop_last = True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [91]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, backbone, finetune=False):
        super().__init__()

        self.backbone = backbone
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = torch.nn.Identity()
        for param in self.backbone.parameters():
          param.requires_grad = False

        self.fca= nn.Linear(num_ftrs, 4)
        self.fcb= nn.Linear(num_ftrs, 88)


    def forward(self, x):
        x_new = self.backbone(x)
        super_out = self.fca(x_new)
        sub_out = self.fcb(x_new)
        return super_out, sub_out

class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.superclass_probs = {i:[] for i in range(4)}
        self.subclass_probs = {i:[] for i in range(88)}

    def train_epoch(self):
        running_loss = 0.0
        device = self.device
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self, append_probs=False):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        device = self.device
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

                super_outputs, sub_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

                if append_probs:
                  probs_super, classes_super = torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1), dim=1)
                  for j, Class in enumerate(classes_super):
                    self.superclass_probs[Class.item()].append(probs_super[j])

                  probs_sub, classes_sub = torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1), dim=1)
                  for j, Class in enumerate(classes_sub):
                    self.subclass_probs[Class.item()].append(probs_sub[j])

            if append_probs:
              self.superclass_probs_mean = {k:torch.mean(torch.tensor(v)) for (k,v) in zip(self.superclass_probs.keys(), self.superclass_probs.values())}
              self.superclass_probs_sigma = {k:torch.std(torch.tensor(v)) for (k,v) in zip(self.superclass_probs.keys(), self.superclass_probs.values())}
              self.subclass_probs_mean = {k:torch.mean(torch.tensor(v)) for (k,v) in zip(self.subclass_probs.keys(), self.subclass_probs.values())}
              self.subclass_probs_sigma = {k:torch.std(torch.tensor(v)) for (k,v) in zip(self.subclass_probs.keys(), self.subclass_probs.values())}
              print("self.superclass_probs_mean: ", self.superclass_probs_mean)
              print("self.superclass_probs_sigma: ", self.superclass_probs_sigma)
              print("self.subclass_probs_mean: ", self.subclass_probs_mean)
              print("self.subclass_probs_sigma: ", self.subclass_probs_sigma)

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')



        # decide on threshold probability cuts

        z = 1.5
        mean_of_superclass_probs_mean = torch.nanmean(torch.tensor([self.superclass_probs_mean[k] for k in self.superclass_probs_mean], dtype=torch.float32))
        mean_of_superclass_probs_sigma = torch.nanmean(torch.tensor([self.superclass_probs_sigma[k] for k in self.superclass_probs_sigma], dtype=torch.float32))
        self.superclass_prob_thresholds = {}
        for i in range(4):
          if not torch.isnan(self.superclass_probs_mean[i]) and not torch.isnan(self.superclass_probs_sigma[i]):
            self.superclass_prob_thresholds[i] = self.superclass_probs_mean[i] - z*self.superclass_probs_sigma[i]
          else:
            self.superclass_prob_thresholds[i] = mean_of_superclass_probs_mean - z*mean_of_superclass_probs_sigma
        print('self.superclass_prob_thresholds: ', self.superclass_prob_thresholds)

        mean_of_subclass_probs_mean = torch.nanmean(torch.tensor([self.subclass_probs_mean[k] for k in self.subclass_probs_mean], dtype=torch.float32))
        mean_of_subclass_probs_sigma = torch.nanmean(torch.tensor([self.subclass_probs_sigma[k] for k in self.subclass_probs_sigma], dtype=torch.float32))
        self.subclass_prob_thresholds = {}
        for i in range(88):
          if not torch.isnan(self.subclass_probs_mean[i]) and not torch.isnan(self.subclass_probs_sigma[i]):
            self.subclass_prob_thresholds[i] = self.subclass_probs_mean[i] - z*self.subclass_probs_sigma[i]
          else:
            self.subclass_prob_thresholds[i] = mean_of_subclass_probs_mean - z*mean_of_subclass_probs_sigma
        print('self.subclass_prob_thresholds: ', self.subclass_prob_thresholds)



        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                super_outputs, sub_outputs = self.model(inputs)

                # if i < 50: #uncomment to get an idea of probabilities
                #   print('softmax super: ', torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1)))
                #   print('softmax super beats cut: ', torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1)) > super_prob_cut)
                #   print('softmax sub: ', torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1)))
                #   print('softmax sub beats cut: ', torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1)) > sub_prob_cut)
                #   print('')
                #   print('')

                super_predicted = (torch.max(super_outputs.data, 1)[1]).item()
                sub_predicted = (torch.max(sub_outputs.data, 1)[1]).item()
                if torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1)) < self.superclass_prob_thresholds[super_predicted]:
                  super_predicted = 3 #novel
                  sub_predicted = 87 #novel
                else:
                  if torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1)) < self.subclass_prob_thresholds[sub_predicted]:
                    sub_predicted = 87 #novel
                test_predictions['subclass_index'].append(sub_predicted)
                test_predictions['superclass_index'].append(super_predicted)


                test_predictions['image'].append(img_name[0])

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('probabilistic_superandsub_test_predictions.csv', index=False)

        if return_predictions:
            return test_predictions

In [92]:
def fine_tune(backbone, cut):
  for i, layer in enumerate(backbone.children()):
    if i < cut: #just train last of the main convolutional layers
      for param in layer.parameters():
        param.requires_grad = False
    else:
      for param in layer.parameters():
        param.requires_grad = True
  return

In [93]:
# Init model and trainer
device = 'cuda'
backbone = models.resnet18(weights='IMAGENET1K_V1')
finetune = False
model = CNN(backbone, finetune=finetune).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [94]:
# Training loop
print('Trainable parameters: ', sum(p.numel() for p in model.parameters() if p.requires_grad))
print('Total parameters: ', sum(p.numel() for p in model.parameters()))
print('----------------------')
print('')
Nepochs = 25
for epoch in range(Nepochs):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    if epoch == Nepochs-1:
      trainer.validate_epoch(append_probs=True)
    else:
      trainer.validate_epoch(append_probs=False)
    print('')
    if epoch == 10:
      fine_tune(trainer.model.backbone, 6)
      print('')
      print('Trainable parameters: ', sum(p.numel() for p in trainer.model.parameters() if p.requires_grad))
      print('Total parameters: ', sum(p.numel() for p in trainer.model.parameters()))
      print('----------------------')
      print('')
      trainer = Trainer(trainer.model, trainer.criterion, trainer.optimizer, trainer.train_loader, trainer.val_loader, trainer.test_loader)
    elif epoch == 20:
      fine_tune(trainer.model.backbone, 5)
      print('')
      print('Trainable parameters: ', sum(p.numel() for p in trainer.model.parameters() if p.requires_grad))
      print('Total parameters: ', sum(p.numel() for p in trainer.model.parameters()))
      print('----------------------')
      print('')
      trainer = Trainer(trainer.model, trainer.criterion, trainer.optimizer, trainer.train_loader, trainer.val_loader, trainer.test_loader)


print('Finished Training')

Trainable parameters:  47196
Total parameters:  11223708
----------------------

Epoch 1
Training loss: 4.905
Validation loss: 4.672
Validation superclass acc: 72.22 %
Validation subclass acc: 19.62 %

Epoch 2
Training loss: 3.737
Validation loss: 4.217
Validation superclass acc: 74.48 %
Validation subclass acc: 27.26 %

Epoch 3
Training loss: 3.297
Validation loss: 4.124
Validation superclass acc: 73.61 %
Validation subclass acc: 24.83 %

Epoch 4
Training loss: 3.051
Validation loss: 3.924
Validation superclass acc: 73.96 %
Validation subclass acc: 30.73 %

Epoch 5
Training loss: 2.890
Validation loss: 4.058
Validation superclass acc: 72.40 %
Validation subclass acc: 30.03 %

Epoch 6
Training loss: 2.720
Validation loss: 3.933
Validation superclass acc: 73.09 %
Validation subclass acc: 29.51 %

Epoch 7
Training loss: 2.633
Validation loss: 3.934
Validation superclass acc: 75.69 %
Validation subclass acc: 31.42 %

Epoch 8
Training loss: 2.545
Validation loss: 3.921
Validation superclas

In [95]:
trainer.model.eval()
predictions = trainer.test(save_to_csv=True, return_predictions=True)

'''
This simple baseline scores the following test accuracy

Superclass Accuracy
Overall: 43.83 %
Seen: 61.11 %
Unseen: 0.00 %

Subclass Accuracy
Overall: 2.03 %
Seen: 9.56 %
Unseen: 0.00 %
'''

self.superclass_prob_thresholds:  {0: tensor(0.8353), 1: tensor(0.8536), 2: tensor(0.8643), 3: tensor(0.8511)}
self.subclass_prob_thresholds:  {0: tensor(0.3666), 1: tensor(0.6700), 2: tensor(0.3309), 3: tensor(0.4409), 4: tensor(0.5270), 5: tensor(0.7537), 6: tensor(0.5436), 7: tensor(0.4859), 8: tensor(0.4078), 9: tensor(0.2070), 10: tensor(0.3593), 11: tensor(0.8184), 12: tensor(-0.0142), 13: tensor(0.2234), 14: tensor(0.3742), 15: tensor(0.5315), 16: tensor(0.3090), 17: tensor(0.5315), 18: tensor(0.2541), 19: tensor(0.7350), 20: tensor(0.5372), 21: tensor(0.4343), 22: tensor(0.2219), 23: tensor(0.3353), 24: tensor(0.7114), 25: tensor(0.0385), 26: tensor(0.4928), 27: tensor(0.5819), 28: tensor(0.8984), 29: tensor(0.4579), 30: tensor(0.5693), 31: tensor(0.3541), 32: tensor(0.3341), 33: tensor(0.4599), 34: tensor(0.9646), 35: tensor(0.4940), 36: tensor(0.5582), 37: tensor(0.4331), 38: tensor(0.3421), 39: tensor(0.6989), 40: tensor(0.7010), 41: tensor(0.5593), 42: tensor(0.4933), 43: t

'\nThis simple baseline scores the following test accuracy\n\nSuperclass Accuracy\nOverall: 43.83 %\nSeen: 61.11 %\nUnseen: 0.00 %\n\nSubclass Accuracy\nOverall: 2.03 %\nSeen: 9.56 %\nUnseen: 0.00 %\n'

In [96]:
predictions

,image,superclass_index,subclass_index
0,0.jpg,1,17
1,1.jpg,0,86
2,2.jpg,2,29
3,3.jpg,3,87
4,4.jpg,1,64
...,...,...,...
12372,12372.jpg,0,6
12373,12373.jpg,2,87
12374,12374.jpg,2,44
12375,12375.jpg,2,47


In [97]:
predictions[predictions['superclass_index']==0]

,image,superclass_index,subclass_index
1,1.jpg,0,86
6,6.jpg,0,24
8,8.jpg,0,87
11,11.jpg,0,87
12,12.jpg,0,85
...,...,...,...
12360,12360.jpg,0,87
12361,12361.jpg,0,87
12362,12362.jpg,0,41
12369,12369.jpg,0,87


In [98]:
predictions['superclass_index'].value_counts()

0    3779
1    3649
2    2907
3    2042
Name: superclass_index, dtype: int64

In [99]:
predictions['subclass_index'].value_counts()

87    4485
52     231
44     222
77     206
64     197
      ... 
67      19
61      18
50      15
66      15
75       7
Name: subclass_index, Length: 88, dtype: int64